In [1]:
from azureml.core import Workspace,Dataset

ws = Workspace.from_config()

dataset_name = 'hdfdata'
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')
# Get a dataset by name
hdf_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)

hdf_ds.to_path()


zf-poc-hp
zf-aml-hp
australiaeast
2bd536a1-23a8-4193-be1e-219890bbd881


['/_2021_06_30_181821#CANape19_log_124.BLF.h5',
 '/_2021_06_30_181923#CANape16_log_008.BLF.h5',
 '/_2021_06_30_182321#CANape19_log_125.BLF.h5',
 '/_2021_06_30_182423#CANape16_log_009.BLF.h5',
 '/_2021_06_30_182821#CANape19_log_126.BLF.h5',
 '/_2021_06_30_182923#CANape16_log_010.BLF.h5',
 '/_2021_06_30_183423#CANape16_log_011.BLF.h5',
 '/_2021_06_30_183821#CANape19_log_128.BLF.h5',
 '/_2021_06_30_183923#CANape16_log_012.BLF.h5',
 '/_2021_06_30_184039#CANape19_log_136.BLF.h5']

In [2]:
experiment_name = 'train-with-hdf'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', 'cpu-cluster')
compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 0)
compute_max_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MAX_NODES', 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU', 'Standard_DS11_v2')


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes, 
                                                                max_nodes=compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. cpu-cluster


In [4]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults
  - torch==1.9.1
  - h5py==3.1.0
  - progressbar>=2.5
  - matplotlib
  - torchvision

Overwriting conda_dependencies.yml


In [5]:
from azureml.core import Environment

pytorch_env = Environment.from_conda_specification(name = 'pytorch-env', file_path = './conda_dependencies.yml')

In [6]:
from azureml.core import ScriptRunConfig
import tempfile
#datastore = ws.get_default_datastore()
#thisdataset = Dataset.File.from_files(path=(datastore, 'hdf_data'))
#print("DATA PATH: " + thisdataset.to_path()->list)
##print("LIST FILES IN DATA PATH...")
#print(os.listdir(thisdataset.to_path()))
hdf_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)
script_folder = os.getcwd()
print(script_folder)
src = ScriptRunConfig(source_directory=script_folder,
                      script='Trainer.py',
                      arguments=['--data_path',hdf_ds.as_mount()],
                      compute_target=compute_target,
                      environment=pytorch_env)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/ttttttt/code/Users/huping1006/ZF_new/scripts


In [7]:
import azureml._restclient.snapshots_client
azureml._restclient.snapshots_client. SNAPSHOT_MAX_SIZE_BYTES =1000000000000

In [8]:
run = exp.submit(src)

Submitting /mnt/batch/tasks/shared/LS_root/mounts/clusters/ttttttt/code/Users/huping1006/ZF_new/scripts directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


In [9]:
model = run.register_model(model_name='pytorch-BD', model_path= 'outputs/thismodel.pt')